In [5]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize, LinearConstraint, Bounds
from colour.colorimetry import SpectralDistribution
from colour.utilities import as_float_array
# colour-0.4.4
from colour.quality import colour_fidelity_index_ANSIIESTM3018, ColourQuality_Specification_ANSIIESTM3018
from colour.colorimetry import sd_to_XYZ
from colour.colorimetry import LMS_ConeFundamentals
from colour.hints import ArrayLike


# --- TM-30-18 Rf, Rg 计算（封装） ---
def compute_tm30_rf_rg(sd_test: SpectralDistribution):
    """
    返回 Rf, Rg
    """
    spec: ColourQuality_Specification_ANSIIESTM3018 = \
        colour_fidelity_index_ANSIIESTM3018(sd_test, additional_data=True)
    return spec.R_f, spec.R_g, spec.CCT, spec.D_uv


# --- mel-DER 计算（基于 CIE S-026 ipRGC 灵敏度） ---
def compute_mel_der(sd_test: SpectralDistribution, sd_ref: SpectralDistribution):
    import colour

    # 获取波长范围
    wavelengths = sd_test.wavelengths
    shape = colour.SpectralShape(wavelengths[0], wavelengths[-1], 1)

    # 获取 melanopic 灵敏度
    try:
        mel_sd = colour.biochemistry.SDS_PHOTORECEPTOR_SENSITIVITIES['Melanopic']
        mel_sd = mel_sd.copy().align(shape)
        mel_sens = mel_sd.values
    except Exception:
        # 如果无法获取，使用高斯近似
        wl_grid = np.array(wavelengths)
        mel_sens = np.exp(-0.5 * ((wl_grid - 490) / 13) ** 2)

    # 获取 V(lambda) 光度权重
    cmfs = colour.MSDS_CMFS['CIE 1931 2 Degree Standard Observer'].copy().align(shape)
    v_lambda = cmfs.values[:, 1]

    # 对齐参考光谱到测试光谱的波长范围
    sd_r = sd_ref.copy().align(shape)

    # 计算 mel-DER
    mel_t = np.trapz(sd_test.values * mel_sens, wavelengths)
    mel_r = np.trapz(sd_r.values * mel_sens, wavelengths)
    v_t = np.trapz(sd_test.values * v_lambda, wavelengths)
    v_r = np.trapz(sd_r.values * v_lambda, wavelengths)

    # mel-DER = (mel_t / v_t) / (mel_r / v_r)
    return float((mel_t / v_t) / (mel_r / v_r))


# --- 读取五通道 SPD ---
def load_channels(path: str, sheets=5) -> (np.ndarray, list[SpectralDistribution]):
    df = pd.read_excel(path, sheet_name=None)
    # 取公共波长
    wls = None
    sds = []
    for name in list(df.keys())[:sheets]:
        d = df[name]
        # 提取波长数值（去掉单位）
        wavelength_str = d['波长'].astype(str)
        wls_numeric = wavelength_str.str.extract(r'(\d+)')[0].astype(float)

        if wls is None:
            wls = wls_numeric.values
        else:
            assert np.allclose(wls, wls_numeric.values), \
                "五路通道波长不一致!"
        # 获取 SPD 列名（除了波长列）
        spd_cols = [col for col in d.columns if col != '波长']
        for col in spd_cols:
            sd = SpectralDistribution(dict(zip(wls, d[col].values)), name=col)
            sds.append(sd)
    return wls, sds


# --- 合成 SPD ---
def mix_spd(weights: ArrayLike, sds: list[SpectralDistribution]) -> SpectralDistribution:
    w = np.array(weights)
    # 确保权重和光谱分布数量匹配
    assert len(w) == len(sds), f"权重数量 {len(w)} 与光谱分布数量 {len(sds)} 不匹配"

    # 获取波长范围
    domain = sds[0].domain

    # 手动计算混合光谱
    mixed_values = np.zeros_like(sds[0].values)
    for i in range(len(sds)):
        mixed_values += w[i] * sds[i].values

    # 创建新的 SpectralDistribution
    sd_mix = SpectralDistribution(dict(zip(domain, mixed_values)), name="Mix")
    return sd_mix


# --- 优化：日间模式 ---
def optimize_day(sds: list[SpectralDistribution], sd_ref: SpectralDistribution):
    # 目标：最大化 Rf → minimize -Rf
    def obj(w):
        sd = mix_spd(w, sds)
        Rf, Rg, CCT, Duv = compute_tm30_rf_rg(sd)
        # 惩罚：Rg 不在 [95,105] 强制离域惩罚
        pen = 0.0
        if not (95 <= Rg <= 105):
            pen += abs(Rg - np.clip(Rg, 95, 105)) * 5
        return -Rf + pen

    # 非线性约束：CCT ∈ [6000,7000]
    def constr_CCT_low(w):
        sd = mix_spd(w, sds)
        return compute_tm30_rf_rg(sd)[2] - 5500

    def constr_CCT_high(w):
        sd = mix_spd(w, sds)
        return 6500 - compute_tm30_rf_rg(sd)[2]

    n = len(sds)
    x0 = np.ones(n) / n
    bounds = Bounds(0, 1)
    lincon = LinearConstraint(np.ones((1, n)), [1], [1])  # 权重和=1
    cons = [
        {'type': 'ineq', 'fun': constr_CCT_low},
        {'type': 'ineq', 'fun': constr_CCT_high}
    ]

    res = minimize(obj, x0, method='SLSQP', bounds=bounds,
                   constraints=[lincon, *cons], options={'ftol': 1e-6})
    w_opt = res.x
    sd_opt = mix_spd(w_opt, sds)
    Rf, Rg, CCT, Duv = compute_tm30_rf_rg(sd_opt)
    mel = compute_mel_der(sd_opt, sd_ref)
    return w_opt, Rf, Rg, CCT, Duv, mel


# --- 优化：夜间模式 ---
def optimize_night(sds: list[SpectralDistribution], sd_ref: SpectralDistribution):
    # 目标：最小化 mel-DER
    def obj(w):
        sd = mix_spd(w, sds)
        mel = compute_mel_der(sd, sd_ref)
        Rf, _, CCT, _ = compute_tm30_rf_rg(sd)
        # 惩罚：Rf < 80
        pen = 0.0
        if Rf < 80:
            pen += (80 - Rf) * 5
        return mel + pen

    def constr_CCT_low(w):
        sd = mix_spd(w, sds)
        return compute_tm30_rf_rg(sd)[2] - 2500

    def constr_CCT_high(w):
        sd = mix_spd(w, sds)
        return 3500 - compute_tm30_rf_rg(sd)[2]

    n = len(sds)
    x0 = np.ones(n) / n
    bounds = Bounds(0, 1)
    lincon = LinearConstraint(np.ones((1, n)), [1], [1])
    cons = [
        {'type': 'ineq', 'fun': constr_CCT_low},
        {'type': 'ineq', 'fun': constr_CCT_high}
    ]
    res = minimize(obj, x0, method='SLSQP', bounds=bounds,
                   constraints=[lincon, *cons], options={'ftol': 1e-6})
    w_opt = res.x
    sd_opt = mix_spd(w_opt, sds)
    Rf, Rg, CCT, Duv = compute_tm30_rf_rg(sd_opt)
    mel = compute_mel_der(sd_opt, sd_ref)
    return w_opt, Rf, Rg, CCT, Duv, mel


# --- 主流程 ---
if __name__ == "__main__":
    path = "Problem 2.xlsx"
    wls, sds = load_channels(path)
    # 参考光源使用 CIE D65
    from colour.colorimetry import SDS_ILLUMINANTS

    sd_ref = SDS_ILLUMINANTS["D65"]

    print("=== 日间模式 (CCT ∈ [5500,6500]) 最优化 ===")
    wd, Rf_d, Rg_d, CCT_d, Duv_d, mel_d = optimize_day(sds, sd_ref)
    print(f"权重分配:")
    print(f"  Blue: {wd[0]:.4f} ({wd[0]*100:.2f}%)")
    print(f"  Green: {wd[1]:.4f} ({wd[1]*100:.2f}%)")
    print(f"  Red: {wd[2]:.4f} ({wd[2]*100:.2f}%)")
    print(f"  Warm White: {wd[3]:.4f} ({wd[3]*100:.2f}%)")
    print(f"  Cold White: {wd[4]:.4f} ({wd[4]*100:.2f}%)")
    print(f"\n性能指标:")
    print(f"  CCT (相关色温): {CCT_d:.1f} K")
    print(f"  Duv (色度偏移): {Duv_d:.4f}")
    print(f"  Rf (色彩保真度指数): {Rf_d:.2f}")
    print(f"  Rg (色彩饱和度指数): {Rg_d:.2f}")
    print(f"  mel-DER (褪黑素抑制等效比): {mel_d:.3f}")
    print(f"\n权重和: {np.sum(wd):.6f}")

    print("\n=== 夜间模式 (CCT ∈ [2500,3500]) 最优化 ===")
    wn, Rf_n, Rg_n, CCT_n, Duv_n, mel_n = optimize_night(sds, sd_ref)
    print(f"权重分配:")
    print(f"  Blue: {wn[0]:.4f} ({wn[0]*100:.2f}%)")
    print(f"  Green: {wn[1]:.4f} ({wn[1]*100:.2f}%)")
    print(f"  Red: {wn[2]:.4f} ({wn[2]*100:.2f}%)")
    print(f"  Warm White: {wn[3]:.4f} ({wn[3]*100:.2f}%)")
    print(f"  Cold White: {wn[4]:.4f} ({wn[4]*100:.2f}%)")
    print(f"\n性能指标:")
    print(f"  CCT (相关色温): {CCT_n:.1f} K")
    print(f"  Duv (色度偏移): {Duv_n:.4f}")
    print(f"  Rf (色彩保真度指数): {Rf_n:.2f}")
    print(f"  Rg (色彩饱和度指数): {Rg_n:.2f}")
    print(f"  mel-DER (褪黑素抑制等效比): {mel_n:.3f}")
    print(f"\n权重和: {np.sum(wn):.6f}")

=== 日间模式 (CCT ∈ [5500,6500]) 最优化 ===
权重分配:
  Blue: 0.1504 (15.04%)
  Green: 0.1707 (17.07%)
  Red: 0.2409 (24.09%)
  Warm White: 0.0096 (0.96%)
  Cold White: 0.4284 (42.84%)

性能指标:
  CCT (相关色温): 5500.0 K
  Duv (色度偏移): 0.0061
  Rf (色彩保真度指数): 92.86
  Rg (色彩饱和度指数): 102.65
  mel-DER (褪黑素抑制等效比): 0.605

权重和: 1.000000

=== 夜间模式 (CCT ∈ [2500,3500]) 最优化 ===
权重分配:
  Blue: 0.2523 (25.23%)
  Green: 0.1055 (10.55%)
  Red: 0.1789 (17.89%)
  Warm White: 0.4633 (46.33%)
  Cold White: 0.0000 (0.00%)

性能指标:
  CCT (相关色温): 2543.4 K
  Duv (色度偏移): -0.0122
  Rf (色彩保真度指数): 80.00
  Rg (色彩饱和度指数): 110.54
  mel-DER (褪黑素抑制等效比): 0.255

权重和: 1.000000
